In [1]:
import pandas as pd
import os

# Load cleaned news
df_news = pd.read_csv("data/processed/cleaned_news.csv")
df_news['date'] = pd.to_datetime(df_news['date']).dt.date
df_sentiment = df_news.groupby('date')['Sentiment'].mean().reset_index()

# List of stock files from data/technical/
stock_dir = "data/technical/"
stock_files = [f for f in os.listdir(stock_dir) if f.endswith("_historical.csv")]

# Calculate correlation for each stock
for file in stock_files:
    symbol = file.split("_")[0]
    df_stock = pd.read_csv(os.path.join(stock_dir, file), parse_dates=["Date"])
    df_stock["Date"] = df_stock["Date"].dt.date
    df_stock["return"] = df_stock["Close"].pct_change()

    df = pd.merge(df_stock, df_sentiment, left_on="Date", right_on="date", how="left")
    df = df.dropna(subset=["return", "Sentiment"])
    
    corr = df["return"].corr(df["Sentiment"])
    print(f"{symbol} correlation: {corr:.4f}")
